# Indirect Zero-Noise-Extrapolation
---

This is our configuration:

```yml
nqubit: 5
layer: 10
connstraint: Flase
```

### Linear regression

The general single variable linear regression: $E_{linear}(\lambda)=c_0 + c_1\lambda$

Multivariate linear regression:

$$E_{linear}(n_R, n_T, nY)=c_0 + c_1 n_R + c_2 n_T + c_3 n_Y$$

Where independent variables are: $n_R, n_T, n_Y$

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import itertools
import os
import pickle
import random
from collections import Counter, defaultdict
from copy import deepcopy
from typing import Any, Callable, Generator, Optional
# Functional programming utilities.
import functools

from sympy import *
init_printing(use_unicode=True)

In [3]:
def get_monomials(n: int, d: int) -> list[str]:
    """Compute monomials of degree `d` in graded lexicographical order."""
    variables = [f"λ_{i}" for i in range(1, n + 1)]
    
    monomials = []
    for degree in range(d, -1, -1):
        # Generate combinations for the current degree
        combos = list(itertools.combinations_with_replacement(variables, degree))
        
        # Sort combinations lexicographically
        combos.sort()
        
        # Construct monomials from sorted combinations
        for combo in combos:
            monomial_parts = []
            counts = Counter(combo)
            # Ensure variables are processed in lexicographical order
            for var in sorted(counts.keys()):
                count = counts[var]
                if count > 1:
                    monomial_parts.append(f"{var}**{count}")
                else:
                    monomial_parts.append(var)
            monomial = "*".join(monomial_parts)
            # Handle the case where degree is 0
            if not monomial:
                monomial = "1"
            monomials.append(monomial)
    # "1" should be the first monomial. Note that order d > c > b > a means vector of monomials = [a, b, c, d].            
    return monomials[::-1]

def sample_matrix(sample_points: list[int], degree: int) -> np.ndarray:
    """Construct a matrix from monomials evaluated at sample points."""
    n = len(sample_points[0])  # Number of variables based on the first sample point
    monomials = get_monomials(n, degree)
    matrix = np.zeros((len(sample_points), len(monomials)))

    for i, point in enumerate(sample_points):
        for j, monomial in enumerate(monomials):
            var_mapping = {f"λ_{k+1}": point[k] for k in range(n)}
            matrix[i, j] = eval(monomial, {}, var_mapping)
    return matrix

def get_eta_coeffs_from_sample_matrix(mat: np.ndarray) -> list[float]:
    """Given a sample matrix compute the eta coefficients."""
    n_rows, n_cols = mat.shape
    if n_rows != n_cols:
        raise ValueError("The matrix must be square.")

    det_m = np.linalg.det(mat)    
    if det_m == 0:
        raise ValueError("The matrix is singular.")
    
    terms = []
    for i in range(n_rows):
        new_mat = mat.copy()
        new_mat[i] = np.array([[0] * (n_cols - 1) + [1]])        
        terms.append(np.linalg.det(new_mat) / det_m)

    return terms

def get_eta_coeffs(lre_scale_factors: list[float], degree: int) -> list[float]:
    """Obtain eta coefficients from list of scale factors and degree."""
    mat = sample_matrix(sample_points=lre_scale_factors, degree=degree)
    return get_eta_coeffs_from_sample_matrix(mat)

def get_eta_coeffs_single_variable(scale_factors: list[float]) -> list[float]:
    """Returns the array of single-variable Richardson extrapolation coefficients associated
    to the input array of scale factors."""
    # Lagrange interpolation formula.
    richardson_coeffs = []
    for l in scale_factors:
        coeff = 1.0
        for l_prime in scale_factors:
            if l_prime == l:
                continue
            coeff *= l_prime / (l_prime - l)
        richardson_coeffs.append(coeff)

    return richardson_coeffs

In [4]:
# sample_points = [ (4, 1, 0), (12, 3, 1), (20, 5, 2), (28, 7, 3), (36, 9, 4), (44, 11, 5), (52, 13, 6), (60, 15, 7), (68, 17, 8), (76, 19, 9), (84, 21, 10), (92, 23, 11), (100, 25, 12), (108, 27, 13), (116, 29, 14), (124, 31, 15), (132, 33, 16), (140, 35, 17), (148, 37, 18), (156, 39, 19)]
# sample_points = [(4, 1, -5.907748909320789, 0),
# (12, 1, -5.916920204491979, 0),
# (12, 3, -5.800746641641066, 1),
# (20, 3, -5.764809805003449, 1),
# (20, 5, -5.679828979633399, 2)
# ] 

In [5]:
n, d = 2, 1
sample_points = [
    (4, 1), (12, 5), (20, 5)
]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")

Monomials: ['1', 'λ_2', 'λ_1']
Monomials: 3


In [6]:
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

⎡1.0  1.0  4.0 ⎤
⎢              ⎥
⎢1.0  5.0  12.0⎥
⎢              ⎥
⎣1.0  5.0  20.0⎦

In [7]:
m = np.linalg.det(mat)

In [8]:
m

In [9]:
from sympy import Matrix

# Define the matrix
m1 = Matrix([
    [1, Symbol('λ1'), Symbol('λ2')],
    [1, 12, 5],
    [1, 20, 5]
])

# Calculate determinant
det1 = m1.det()

print(det1/m)


0.25*λ2 - 1.25


In [10]:
# Define the matrix
m2 = Matrix([
    [1, 4, 1],
    [1, Symbol('λ1'), Symbol('λ2')],
    [1, 20, 5]
])

# Calculate determinant
det2 = m2.det()

print(det2/m)

0.125*λ1 - 0.5*λ2


In [11]:
m3 = Matrix([
    [1, 4, 1],
    [1, 12, 5],
    [1, Symbol('λ1'), Symbol('λ2')]
])

# Calculate determinant
det3 = m3.det()

print(det3/m)

-0.125*λ1 + 0.25*λ2 + 0.25


In [12]:
print((det1/32)+(det2/32)+(det3/32))

-1


In [13]:
n =2
d= 1
sample_points = [(0,0), (0,1), (1,1)]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

Monomials: ['1', 'λ_2', 'λ_1']
Monomials: 3


⎡1.0   0    0 ⎤
⎢             ⎥
⎢1.0  1.0   0 ⎥
⎢             ⎥
⎣1.0  1.0  1.0⎦

In [14]:
get_eta_coeffs_from_sample_matrix(mat)

In [17]:
n, d = 2, 2
sample_points = [
    (4, 1), (12, 7), (36, 5), (36, 13), (52, 5), (52, 13)
]
monomials = get_monomials(n=n, d=d)
print(f"Monomials: {monomials}")
print(f"Monomials: {len(monomials)}")



Monomials: ['1', 'λ_2', 'λ_1', 'λ_2**2', 'λ_1*λ_2', 'λ_1**2']
Monomials: 6


In [18]:
mat = sample_matrix(sample_points=sample_points, degree=d)
Matrix(mat)

⎡1.0  1.0   4.0    1.0    4.0    16.0 ⎤
⎢                                     ⎥
⎢1.0  7.0   12.0  49.0   84.0   144.0 ⎥
⎢                                     ⎥
⎢1.0  5.0   36.0  25.0   180.0  1296.0⎥
⎢                                     ⎥
⎢1.0  13.0  36.0  169.0  468.0  1296.0⎥
⎢                                     ⎥
⎢1.0  5.0   52.0  25.0   260.0  2704.0⎥
⎢                                     ⎥
⎣1.0  13.0  52.0  169.0  676.0  2704.0⎦

In [34]:
A = np.linalg.det(mat)

In [20]:
get_eta_coeffs_from_sample_matrix(mat)

$ E = c_1 + c_2 n_T + c_3 n_R + c_4 n^{2}_{T} + c_5 n_R n_T + c_6 n^{2}_T$

In [ ]:
(4, 1, -5.942817754597376), (12, 7, 5.610630688805676), (36, 5, -5.61959508599682), (36, 13, -5.267674373577144), (52, 5, -5.540521346789942), (52, 13, -5.135264624148199)

In [35]:
A

In [43]:
mat

array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

In [52]:
A = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M1 = np.array([[1, 0, 0, 0, 0, 0],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])



M2 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M3 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M4 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M5 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1, 0, 0, 0, 0, 0],
       [1.000e+00, 1.300e+01, 5.200e+01, 1.690e+02, 6.760e+02, 2.704e+03]])

M6 = np.array([[1.000e+00, 1.000e+00, 4.000e+00, 1.000e+00, 4.000e+00, 1.600e+01],
       [1.000e+00, 7.000e+00, 1.200e+01, 4.900e+01, 8.400e+01, 1.440e+02],
       [1.000e+00, 5.000e+00, 3.600e+01, 2.500e+01, 1.800e+02, 1.296e+03],
       [1.000e+00, 1.300e+01, 3.600e+01, 1.690e+02, 4.680e+02, 1.296e+03],
       [1.000e+00, 5.000e+00, 5.200e+01, 2.500e+01, 2.600e+02, 2.704e+03],
       [1, 0, 0, 0, 0, 0]])

In [55]:
Matrix(M6)

⎡1.0  1.0   4.0    1.0    4.0    16.0 ⎤
⎢                                     ⎥
⎢1.0  7.0   12.0  49.0   84.0   144.0 ⎥
⎢                                     ⎥
⎢1.0  5.0   36.0  25.0   180.0  1296.0⎥
⎢                                     ⎥
⎢1.0  13.0  36.0  169.0  468.0  1296.0⎥
⎢                                     ⎥
⎢1.0  5.0   52.0  25.0   260.0  2704.0⎥
⎢                                     ⎥
⎣1.0   0     0      0      0      0   ⎦

In [61]:
detA = np.linalg.det(mat)

In [62]:
detA

In [67]:
RE = (-5.942817754597376 * np.linalg.det(M1)/detA) + (-5.610630688805676*np.linalg.det(M2)/detA) + (-5.61959508599682 * np.linalg.det(M3)/detA) + (-5.267674373577144  * np.linalg.det(M4)/detA) + (-5.540521346789942 * np.linalg.det(M5)/detA) + (-5.135264624148199 * np.linalg.det(M6)/detA)

In [68]:
RE

In [70]:
a = -6.00752391934362
b = -6.026674183332272

In [72]:
b<a

True

In [73]:
import numpy as np

# Coefficients and matrices
coefficients = [-5.942817754597376, -5.610630688805676, -5.61959508599682, -5.267674373577144, -5.540521346789942, -5.135264624148199]
matrices = [M1, M2, M3, M4, M5, M6]  # Assuming M1, M2, ..., M6 are defined

# Calculate detA
detA = np.linalg.det(A)  # Assuming A is defined

# Initialize result
RE = 0

# Loop over coefficients and matrices
for coeff, matrix in zip(coefficients, matrices):
    RE += coeff * np.linalg.det(matrix) / detA

print("Result:", RE)


Result: -6.007523919343623
